# Lab 1. Wybrane zagadnienia programowania obiektowego w Pythonie.

## 1. Cykl życia obiektu.

**Listing 1**

In [19]:
class Dummy():
    # a to poniżej to docstring
    """ Testowy obiekt """

    # zmienna klasowa - współdzielona pomiędzy instancje klasy Dummy
    class_counter = 0

    # inicjalizator
    def __init__(self):
        print("__init__()")
        # dostęp do zmiennej klasowej odbywa się jak poniżej
        Dummy.class_counter += 1
        # zmienna instancji
        self.foo = 'foo foo'

    # metoda new() jest wywoływana przed inicjalizatorem
    # i zwraca obiekt wywoływanego typu (czyli tu Dummy)
    # można to nazwać konstruktorem
    def __new__(cls, *args, **kwargs):
        print('__new__()')
        return super().__new__(cls)

    # metoda del jest wywoływana w momencie usuwania obiektu z pamięci
    # czyli w momencie "śmierci" obiektu (spadek liczby referencji do 0 do obszaru
    # w pamięci zajmowanego przez obiekt), nazywana destruktorem
    def __del__(self):
        print('__del__()')


d = Dummy()
print(d.__doc__)
print(d.class_counter)
print(d.foo)

# zmienna obiektu
d.so_what = 'Who am I?'

print(d.__dict__)

del d

d1 = Dummy()
print(d1.class_counter)
print(d1.foo)
print(d1.__dict__)


__new__()
__init__()
Testowy obiekt 
1
foo foo
{'foo': 'foo foo', 'so_what': 'Who am I?'}
__del__()
__new__()
__init__()
__del__()
2
foo foo
{'foo': 'foo foo'}


**Listing 2**

In [ ]:
# czy to jest poprawny składniowo kod Python?
class whoops: pass

wh = whoops()

# a to?
class whoopsi: ...


# podobny do powyższego zapis znajdziemy w plikach z rozszerzeniem .pyi, które są plikami zawierającymi
# tylko szkielet (ang. stub), który może zawierać tylko sygnatury funkcji, klas i ich metod
# które pełnią funkcję interfejsów w Pythonie, których nie możemy implementować w sposób znany chociażby
# z języka Java czy C#. Można to np. zobaczyć w pliku builtins.pyi, ale i wielu innych


wh2 = whoopsi()
print(wh.__class__, wh2.__class__)


<class '__main__.whoops'> <class '__main__.whoopsi'>


Więcej o wyrażeniu `pass`: https://docs.python.org/3/tutorial/controlflow.html#pass-statements

Więcej o wyrażeniu `...` (Ellipsis): https://docs.python.org/3/library/constants.html#Ellipsis

**Listing 3** - to tylko prezentacja pewnej idei, ale nie jest to ani kompletna, ani zalecana metoda implementacji wzorca Singleton w Pythonie. Nie rób tego w domu!

In [ ]:
# poniższy przykład prezentuje jak można osiągnąć niezbyt funkcjonalna i nie kompletną (np. nie ma obsługi atrybutów konstruktora)
# implementację wzorca singleton, czyli ograniczenie możliwości tworzenia instancji danego obiektu tylko do 1 sztuki
class DummySingleton():
    """ Testowy obiekt """

    instance = None

    def __init__(self):
        print("__init__()")

    def __new__(cls):
        print('__new__()')
        if DummySingleton.instance is None:
            DummySingleton.instance = object.__new__(cls)
            
        return DummySingleton.instance

    def __del__(self):
        print('__del__()')

single = DummySingleton()
single.name = "To Singleton"
print(id(single))

single2 = DummySingleton()
print(single2.name)
print(id(single2))

__new__()
__init__()
2120635527728
__new__()
__init__()
To Singleton
2120635527728


## 2. Wybrane metody magiczne modelu danych Pythona.

Poniżej przedstawione zostaną przykłady działania wybranych metod magicznych modelu danych Pythona. Niektóre z nich zostana również zaprezentowane w działaniu poprzez ich przeciążenie (w odniesieniu do Pythona używa się często terminu **przeciążanie operatorów**) we własnych definicjach klas.  

**Listing 4**

In [ ]:
class Square:
    """ Klasa do obsługi figury typu kwadrat"""

    def __init__(self, side=1):
        self.side = side
    
    def __str__(self) -> str:
        return f"Square({self.side})"
    
    # tu możemy się zastanowić jak zrealizować dodawanie dwóch kwadratów
    # czy sumujemy długość boku (stworzymy kwadrat o boku, który opisuje dwa kwadraty?, czy też sumujemy pole i wyznaczamy nową wartość size?
    # przyjmijmy ten drugi scenariusz
    # nazwa pierwszego argumenty metody, która w dokumentacji zawsze nosi nazwę self, jest tylko umową, ale wymogiem jest to, że będzie przekazywana
    # do metody zawsze jako pierwszy argument. Więc jak wolimy this to ...
    def __add__(this, other):
        import math
        if isinstance(other, type(this)):
            new_side = math.sqrt(this.side**2 + other.side**2)
            return type(this)(new_side)
        else:
            raise TypeError(
                "unsupported operand for +: "
                f"'{type(this).__name__}' and '{type(other).__name__}'"
            )
    
    def __mul__(self, scale: int | float):
        return Square(self.side * scale)
    
    def __truediv__(self, scale: int | float):
        return Square(self.side / scale)
    
    def __eq__(self, other):
        if isinstance(other, Square):
        # lub
        # if isinstance(other, type(self)):
            return self.side == other.side
        return False


if __name__ == '__main__':
    s = Square(2)
    s2 = Square(3)

    print(s, ' + ', s2, ' to ', s + s2)
    print(s * 3)
    print(s / 2)
    print(s == s2)

False
True
Square(2)  +  Square(3)  to  Square(3.605551275463989)
Square(6)
Square(1.0)
False


**Listing 5**

Ten listing przedstawia trzy sposoby na implementację klasy z abstrakcyjnymi metodami. Metody abstrakcyjne to takie, które w miejscu ich zdefiniowania nie poasiadają ciała (implementacji) i należy ją zapewnić w klasie potomnej (czyli dziedziczącej po niej).

In [ ]:
# sposób 1 - wyrzucanie wyjątku NotImplementedError

class Figure:

    def __init__(self):
        raise NotImplementedError("Należy zaimplementować tę metodę")
    def get_area(self): 
        raise NotImplementedError("Należy zaimplementować tę metodę")

class CircleEmpty(Figure):
    pass

class Circle(Figure):

    def __init__(self, radius):
        self.radius = radius

if __name__ == "__main__":
    c = Circle(5)
    c2 = CircleEmpty()

NotImplementedError: Należy zaimplementować tę metodę

In [ ]:
# sposób 2 - użycie instrukcji assert

class Figure:

    def __init__(self):
        assert False, "Należy zaimplementować tę metodę"
    def get_area(self): 
        assert False, "Należy zaimplementować tę metodę"

class CircleEmpty(Figure):
    pass

class Circle(Figure):

    def __init__(self, radius):
        self.radius = radius

if __name__ == "__main__":
    c = Circle(5)
    c2 = CircleEmpty()

AssertionError: Należy zaimplementować tę metodę

In [ ]:
# sposób 3 - z użyciem klas ABC (Abstract Base Classes) - tutaj tylko prosty przykład, bardziej szczegółowo
# klasami ABC zajmiemy się na kolejnych laboratoriach
from abc import ABCMeta, abstractmethod

class Figure(metaclass=ABCMeta):

    @abstractmethod
    def __init__(self): pass

    @abstractmethod
    def get_area(self): ...

class CircleEmpty(Figure):
    pass

class Circle(Figure):

    def __init__(self, radius):
        self.radius = radius

    def get_area(self):
        import math
        return (math.PI * self.radius) ** 2

if __name__ == "__main__":
    c = Circle(5)
    c2 = CircleEmpty()

TypeError: Can't instantiate abstract class CircleEmpty without an implementation for abstract methods '__init__', 'get_field'

W poniższym przykładzie wykorzystamy magiczne metody, które są częściej wykorzystywane dla kolekcji.  

**Listing 6**

In [ ]:
class Field:

    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        return f"Field({self.value})"
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return self.value == other.value
        else:
            return False


class KoloFortuny:

    def __init__(self, *fields):
        self.fields = [tuple[int, Field]]
        if fields:
            self.fields = list(zip(range(1, len(fields) + 1), [Field(val) for val in fields]))

    def __str__(self):
        return f"KoloFortuny({self.fields})"
    
    def __getitem__(self, idx):
        return self.fields[idx][1]
    
    def __setitem__(self, idx, val:Field):
        if isinstance(val, Field):
            self.fields[idx] = (idx + 1, val)
        else:
            raise TypeError("Można wstawić tylko wartość typu Field!")
    
    def __iter__(self):
        return iter(self.fields)
    
    def __contains__(self, other):
        if isinstance(other, Field):
            return other in [val for _, val in self.fields]
        else:
            raise TypeError("Nie umiem znaleźć innych elementów niż te typu Field!")


if __name__ == '__main__':

    kolo = KoloFortuny(100, 'Bankrut', 50, 300, 'Niespodzianka', 1000)
    print(kolo)
    print(kolo[3])
    kolo[3] = Field(750)
    # odkomentuj poniższą linię i sprawdź działanie
    # kolo[3] = 750

    for field in kolo:
        print(field)
    
    print(Field(750) in kolo)




KoloFortuny([(1, Field(100)), (2, Field(Bankrut)), (3, Field(50)), (4, Field(300)), (5, Field(Niespodzianka)), (6, Field(1000))])
Field(300)
(1, Field(100))
(2, Field(Bankrut))
(3, Field(50))
(4, Field(750))
(5, Field(Niespodzianka))
(6, Field(1000))
True
()
{'imie': 'Marek', 'nazwisko': 'Malinowski'}
(1, 2, 3, 6, 34, 5)
{}
Adam


## Zadania do wykonania


1. W klasie `Square`:
   * dziedziczenie po klasie `Figure` - pamiętaj o implementacji wymaganych metod
   * dodaj przeciążony operator `__iadd__` i zaiplementuj jego działanie analogicznie do logiki w metodzie `__add__`,
   * dodaj przeciążony operator `__radd__`, ale powinien działać poprawnie tylko dla dodawania do instancji klasy `Square` wartości typu `int`
   * zmodyfikuj kod metod `__iadd__` oraz `__add__`, aby dla typów `int` działały tak samo jak `__radd__`
   * dodaj odpowiedni kod testujący powyższe funkcjonalności

2. W klasie `Figure`:
   *  dodaj abstrakcyjną metodę `get_circumference` (zwracanie obwodu figury) i zaimplementuj je w klasie `Circle` oraz `Square`. Zapisz kod testujący poprawne działanie tej funkcjonalności,
   *  zastanów się czy możliwe jest przeciążenie operatorów `__gt__` oraz `__lt__` w tej klasie tak, aby porównywane było pole figur? Zaimplementuj odpowiednie rozwiązanie.
  
3. W klasie `Field` zaimplementuj:
   * przeciążenie operatora `__setattr__` dla atrybutu `value` z poniższymi zasadami:
     * jeżeli ustawiamy wartość tego atrybutu jako typ `int` to możliwe jest przypisanie wartości z zakresu 10 - 2000
     * jeżeli jest to typ `str` to można przypisać cokolwiek
     * w innym przypadku rzucamy wyjątek `TypeError`
     * napisz kod testujący te funkcjonalności.


## Ciekawe informacje o Pythonie w kontekście "zaawansowane":

* https://leanpub.com/insidethepythonvirtualmachine/read
* https://github.com/python/cpython
* https://www.geeksforgeeks.org/python-virtual-machine/
* https://www.geeksforgeeks.org/python-vs-cpython/